In [1]:
import numpy as np
import pandas as pd
import json
import codecs
import datetime
import os
import sys
sys.path.append('.')
import matplotlib as mpl
import matplotlib.pyplot as plt
from vector_creator.raw_to_df.rawdata_to_df import *
from vector_creator.score_vectors.score_vectors_assembly import *
from vector_creator.stats_models.entropy import *
from vector_creator.preprocess.est_by_df_column import *
from vector_creator.stats_models.auto_regression import *
from vector_creator.preprocess.install_apps_features import *
from vector_creator.stats_models.estimators import *
from vector_creator.score_vectors.vector_indexer import vector_desc_call_logs
from vector_creator.score_vectors.vector_descriptor import *
path = '/Users/smuel/AndroidStudioProjects/LuckyMeIntgratedSDK/metadata_init_phase/'

In [2]:
rdata = list_of_json_files(path)
rdata[0][0]

'05fa68e5-0ba3-4783-9c07-42740c4914c6'

In [3]:
raw = group_metadata(path)
uid  = '05fa68e5-0ba3-4783-9c07-42740c4914c6'
file = '05fa68e5-0ba3-4783-9c07-42740c4914c6_11-06-2021_09-53-01.json'
raw_data = json.load(codecs.open(path + file, 'r', 'utf-8-sig'))

In [4]:
raw_data.keys()

dict_keys(['SamplingConfigurations', 'CallLogs', 'ImgMetaData', 'InstallApps', 'LocationInfo', 'BatteryInfo', 'WifiInfo', 'ScreenInfo'])

In [5]:
loc, df_r = create_df_from_init_metadata(uid, raw_data)

In [6]:
df_r.keys()

dict_keys(['05fa68e5-0ba3-4783-9c07-42740c4914c6_SamplingConfigurations', '05fa68e5-0ba3-4783-9c07-42740c4914c6_CallLogs', '05fa68e5-0ba3-4783-9c07-42740c4914c6_ImgMetaData', '05fa68e5-0ba3-4783-9c07-42740c4914c6_InstallApps', '05fa68e5-0ba3-4783-9c07-42740c4914c6_LocationInfo', '05fa68e5-0ba3-4783-9c07-42740c4914c6_BatteryInfo', '05fa68e5-0ba3-4783-9c07-42740c4914c6_WifiInfo', '05fa68e5-0ba3-4783-9c07-42740c4914c6_ScreenInfo'])

In [7]:
df=df_r['05fa68e5-0ba3-4783-9c07-42740c4914c6_ImgMetaData']

In [8]:
df.head(5)

IMAGE_TYPE     IMAGE_DATE_TIME
0  image/jpeg 2019-10-16 08:48:55
1  image/jpeg 2019-10-16 08:49:06
2  image/jpeg 2019-10-16 08:49:07
3  image/jpeg 2019-10-20 15:42:48
4  image/jpeg 2019-10-22 14:09:27

In [9]:
loc_tuple = (loc[0]['Latitude'], loc[0]['Longitude'])

In [10]:
x = score_vector_constructor(path, 'photo-gallery')

-> processed -> 724
-> processed -> 1448
-> processed -> 1558
-> processed -> 420
-> processed -> 1278
-> processed -> 311
-> processed -> 462
-> processed -> 3294
-> processed -> 1436
-> processed -> 1369
-> processed -> 2398
-> data to small to process -> 54
-> processed -> 607
-> processed -> 553
-> data to small to process -> 193
-> processed -> 204
-> data to small to process -> 137
-> processed -> 1526
-> processed -> 964
-> processed -> 897
-> data to small to process -> 28
(17, 33)


In [42]:
from scipy.stats import zscore
from sklearn import preprocessing

In [18]:
#read the tags from json file for user register non register and create df with tag column
def create_tag_df(json_file_path, column_name): # 'entries.json'
    tag_data = json.load(codecs.open(json_file_path, 'r', 'utf-8-sig'))
    df_tag = pd.DataFrame(tag_data)
    df_tag = df_tag.rename(columns={column_name: ''}) # 'app_userid'
    dftag = df_tag.set_index('')
    return dftag

In [68]:
def norm_df(old_df):
    inxs = old_df.index.values
    zs = zscore(old_df.values, axis=0, ddof=1)
    return pd.DataFrame(zs, columns=old_df.columns, index=inxs)
n_df = norm_df(x)

In [44]:
def norm_df_2(df):
    inxs = df.index.values
    std_scale = preprocessing.StandardScaler().fit(df)
    df_std = std_scale.transform(df)
    return pd.DataFrame(df_std, columns=df.columns, index=inxs)
n2_df = norm_df_2(x)

In [52]:
len(n2_df)

17

0.970142500145332

In [24]:
dftag = create_tag_df('entries.json')
dftag

User_Register
                                                  
005b6c49-c4ed-438b-9092-ad02ed4d672e         false
005d6e1d-7a19-422f-a71a-1555b6ca5724         false
007a0785-1321-4112-be89-e6549a4725a2          true
008e4a51-2ded-4a7f-a822-e1a4756d7ebd         false
00c86b27-f076-4560-bb3d-2fb0364b23a1         false
...                                            ...
ff992f28-a57d-400d-87eb-5249fdc84337          true
ffc7b3ed-d66e-4075-ba35-aa1b96c88db7         false
ffccdd4d-03a1-41ba-bfcf-5f1428db631b         false
ffe09b02-12df-4551-9d88-3f7f52707d57          true
fff4c564-d0f5-4358-9d34-0a15da2d588e          true

[2976 rows x 1 columns]

In [30]:
len(dftag)

2976

In [71]:
dfjoin = n_df.join(dftag)
dfjoin['User_Register'] = dfjoin['User_Register'].fillna('false')

In [72]:
len(dfjoin)

17

In [73]:
dfjoin.head(5)

median number photos taken in week - location  \
05fa68e5-0ba3-4783-9c07-42740c4914c6                                      -0.341279   
12fa7561-2e77-4334-b9e1-6858ed27570f                                      -0.501170   
1362474c-0fb5-4932-a33d-5a30f23ac7e3                                      -0.501170   
148b0326-09e2-404f-9de6-aff6d44f2a38                                       0.138393   
17ac0e20-b292-492c-9adc-faf60a068926                                      -0.501170   

                                      mad number photos taken in week - scale  \
05fa68e5-0ba3-4783-9c07-42740c4914c6                                -0.241303   
12fa7561-2e77-4334-b9e1-6858ed27570f                                -0.676379   
1362474c-0fb5-4932-a33d-5a30f23ac7e3                                -0.676379   
148b0326-09e2-404f-9de6-aff6d44f2a38                                -0.054842   
17ac0e20-b292-492c-9adc-faf60a068926                                -0.676379   

                                      sample weeks ratio  \
05fa68e5-0ba3-4783-9c07-42740c4914c6            0.228388   
12fa7561-2e77-4334-b9e1-6858ed27570f           -1.032462   
1362474c-0fb5-4932-a33d-5a30f23ac7e3           -1.168924   
148b0326-09e2-404f-9de6-aff6d44f2a38            1.206315   
17ac0e20-b292-492c-9adc-faf60a068926           -1.123054   

                                      mean number of photos in burst  \
05fa68e5-0ba3-4783-9c07-42740c4914c6                       -0.358423   
12fa7561-2e77-4334-b9e1-6858ed27570f                        2.112833   
1362474c-0fb5-4932-a33d-5a30f23ac7e3                       -0.264540   
148b0326-09e2-404f-9de6-aff6d44f2a38                       -0.379527   
17ac0e20-b292-492c-9adc-faf60a068926                        2.298451   

                                      mean number of bursts a week  \
05fa68e5-0ba3-4783-9c07-42740c4914c6                     -0.519930   
12fa7561-2e77-4334-b9e1-6858ed27570f                     -0.751947   
1362474c-0fb5-4932-a33d-5a30f23ac7e3                     -0.451051   
148b0326-09e2-404f-9de6-aff6d44f2a38                     -0.041939   
17ac0e20-b292-492c-9adc-faf60a068926                     -0.832466   

                                      median night time number photos  \
05fa68e5-0ba3-4783-9c07-42740c4914c6                        -0.543800   
12fa7561-2e77-4334-b9e1-6858ed27570f                        -0.543800   
1362474c-0fb5-4932-a33d-5a30f23ac7e3                        -0.543800   
148b0326-09e2-404f-9de6-aff6d44f2a38                         0.350839   
17ac0e20-b292-492c-9adc-faf60a068926                        -0.543800   

                                      mad night time number photos  \
05fa68e5-0ba3-4783-9c07-42740c4914c6                     -0.669567   
12fa7561-2e77-4334-b9e1-6858ed27570f                     -0.669567   
1362474c-0fb5-4932-a33d-5a30f23ac7e3                     -0.669567   
148b0326-09e2-404f-9de6-aff6d44f2a38                      0.572176   
17ac0e20-b292-492c-9adc-faf60a068926                     -0.669567   

                                      sampled week nights ratio  \
05fa68e5-0ba3-4783-9c07-42740c4914c6                  -0.378252   
12fa7561-2e77-4334-b9e1-6858ed27570f                  -0.895632   
1362474c-0fb5-4932-a33d-5a30f23ac7e3                  -1.021261   
148b0326-09e2-404f-9de6-aff6d44f2a38                   1.281879   
17ac0e20-b292-492c-9adc-faf60a068926                  -1.115439   

                                      entropy on number of photos taken in night time  \
05fa68e5-0ba3-4783-9c07-42740c4914c6                                         0.252810   
12fa7561-2e77-4334-b9e1-6858ed27570f                                        -0.401850   
1362474c-0fb5-4932-a33d-5a30f23ac7e3                                         0.667828   
148b0326-09e2-404f-9de6-aff6d44f2a38                                         0.032790   
17ac0e20-b292-492c-9adc-faf60a068926                                        -1.636334   

               

In [28]:
len(dfjoin[dfjoin.User_Register == 'false'])

13

In [78]:
dfjoin.values[:,4].std()

0.9701425001453321